Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Prepare)

Today's guided module project will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills. The competition will begin

## Learning Objectives
* <a href="#p0">Part 0</a>: Kaggle Competition
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy

# Text Feature Extraction & Classification Pieplines (Learn)
<a id="p1"></a>

## Overview

Sklearn pipelines allow you to stitch together multiple components of a machine learning process. The idea is that you can pass you raw data and get predictions out of the pipeline. This ability to pass raw input and receive a prediction from a singular class makes pipelines well suited for production, because you can pickle a a pipeline without worry about other data preprocessing steps. 

In [2]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
# Dataset
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism',
              'talk.religion.misc']

data = fetch_20newsgroups(subset='train', categories=categories)

In [4]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

In [5]:
# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [6]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   27.7s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [7]:
grid_search.best_score_

0.8938156359393232

In [8]:
grid_search.predict(['Send me lots of money now', 'you won the lottery in Nigeria'])

array([1, 1], dtype=int64)

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [3]:
import pandas as pd
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

### Define Pipeline Components

In [5]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect),
                 ('clf', clf)])

In [18]:
train.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [6]:
parameters = {
    'vect__max_df': (0.75, .98, 1.0),
    'vect__min_df': (.02, .05, .1),
    'vect__max_features': (50 , 250, 500, 1000, 1500),
    'clf__n_estimators':(5, 10, 15, 20),
    'clf__max_depth':(1, 5,10,15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train.description, train.category)

Fitting 3 folds for each of 900 candidates, totalling 2700 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.1s


KeyboardInterrupt: 

In [48]:
grid_search.best_score_

AttributeError: 'GridSearchCV' object has no attribute 'best_score_'

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [27]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [29]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [30]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [31]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission2.csv', index=False)

## Challenge

You're trying to achienve 90% Accuracy on your model.

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Overview

In [8]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=100, 
                   algorithm='randomized',
                   n_iter=10)

In [9]:
params = { 
    'lsi__svd__n_components': [10,100,250],
    'lsi__vect__max_df':[.9, .95, 1.0],
    'clf__n_estimators':(5, 10,20)
    
}

In [10]:
# LSI
lsi = Pipeline([('vect', vect), ('svd', svd)])


# Pipe
pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

NameError: name 'rfc' is not defined

In [37]:
# Fit
grid_search = GridSearchCV(pipe,params, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data.data, data.target)

Fitting 5 folds for each of 27 candidates, totalling 135 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.3min


KeyboardInterrupt: 

In [14]:
grid_search.best_score_

0.8786464410735122

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [11]:
from xgboost import XGBClassifier
lsi = Pipeline([('vect', vect), ('svd', svd)])
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
clf = XGBClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', clf)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [13]:
parameters = {
    'lsi__svd__n_components': [250, 300],
    'lsi__vect__max_df': [.93, .95, .98],
    'clf__max_depth':(3, 5, 10),
    'clf__tree_method': ['gpu_exact'],
    'clf__num_leaves': (6, 50),
    'clf__min_child_samples': (100, 500),
    'clf__min_child_weight': [1e-5, 1, 1e4]
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=1, verbose=10)
grid_search.fit(train.description, train.category)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.93 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.93, score=0.9223638470451911, total=   8.8s
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.93 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    9.0s remaining:    0.0s


[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.93, score=0.9304750869061413, total=   8.3s
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.93 


...

[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=250, lsi__vect__max_df=0.98, score=0.8976744186046511, total=   9.1s
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  1.4min remaining:    0.0s


[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93, score=0.9212050984936269, total=   9.7s
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93 
[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93, score=0.9269988412514485, total=  10.2s
[CV] clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93 
[CV]  clf__max_depth=3, clf__min_child_samples=100, clf__min_child_weight=1e-05, clf__num_leaves=6, clf__tree_method=gpu_exact, lsi__svd__n_components=300, lsi__vect__max_df=0.93, score=0.90116279069767

[Parallel(n_jobs=1)]: Done 648 out of 648 | elapsed: 151.3min finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
       fit_params=None, iid='warn', n_jobs=1,
       param_grid={'lsi__svd__n_components': [250, 300], 'lsi__vect__max_df': [0.93, 0.95, 0.98], 'clf__max_depth': (3, 5, 10), 'clf__tree_method': ['gpu_exact'], 'clf__num_leaves': (6, 50), 'clf__min_child_samples': (100, 500), 'clf__min_child_weight': [1e-05, 1, 10000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=10)

In [14]:
grid_search.best_score_

0.9203402938901779

In [15]:
grid_search.best_params_

{'clf__max_depth': 3,
 'clf__min_child_samples': 100,
 'clf__min_child_weight': 1e-05,
 'clf__num_leaves': 6,
 'clf__tree_method': 'gpu_exact',
 'lsi__svd__n_components': 250,
 'lsi__vect__max_df': 0.95}

### Make a Submission File
*Note:* You are only allowed two submissions a day. Only submit if you feel you cannot achieve higher test accuracy. 

In [16]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [17]:
submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [18]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,1
3,1024,1
4,1902,1


In [19]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv('./data/submission4.csv', index=False)

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

# Overview

In [19]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [20]:
doc = nlp("Two bananas in pyjamas")

In [21]:
bananas_vector = doc.vector
print(len(bananas_vector))

300


In [26]:
len(X) == len(data.data)

True

In [22]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [23]:
X = get_word_vectors(data.data)

rfc.fit(X, data.target)

/Users/jonathansokoll/anaconda3/envs/U4-S1-NLP-DS6/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [27]:
rfc.score(X, data.target)

0.9918319719953326

## Follow Along

In [ ]:
# Apply to your Dataset

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Review

To review this module: 
* Continue working on the Kaggle comeptition
* Find another text classification task to work on